In [1]:
#Prints **all** console output, not just last item in cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

**Eric Meinhardt / emeinhardt@ucsd.edu**

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Motivation" data-toc-modified-id="Motivation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Motivation</a></span></li><li><span><a href="#Load-data" data-toc-modified-id="Load-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load data</a></span><ul class="toc-item"><li><span><a href="#Combinatoric-summary" data-toc-modified-id="Combinatoric-summary-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Combinatoric summary</a></span></li></ul></li><li><span><a href="#Identifying-a-possible-Boolean-concept" data-toc-modified-id="Identifying-a-possible-Boolean-concept-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Identifying a possible Boolean concept</a></span></li><li><span><a href="#General-case" data-toc-modified-id="General-case-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>General case</a></span><ul class="toc-item"><li><span><a href="#Demo" data-toc-modified-id="Demo-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Demo</a></span></li><li><span><a href="#General-case-redux:-BFE-comparison" data-toc-modified-id="General-case-redux:-BFE-comparison-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>General case redux: BFE comparison</a></span></li></ul></li><li><span><a href="#Functions" data-toc-modified-id="Functions-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Functions</a></span><ul class="toc-item"><li><span><a href="#Motivation" data-toc-modified-id="Motivation-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Motivation</a></span></li><li><span><a href="#Function-combinatorics" data-toc-modified-id="Function-combinatorics-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Function combinatorics</a></span></li></ul></li></ul></div>

# Motivation

In this notebook we show `prague` can be used to identify feature vectors compatible with (or that exactly pick out) an observed set of objects, using phonology as a motivating example.

Note that on the current test dataset (`brh.py`), the machine running this notebook will probably need (at peak) 20-30GB of RAM.

# Load data

In [2]:
%cd ..

/Users/emeinhardt/Repositories/prague


In [3]:
from funcy import *
from functools import reduce

In [4]:
import numpy as np

In [5]:
import prague

In [6]:
%ls data

bakovic_chart_riggle_hayes.tsv
bakovic_chart_riggle_hayes_remapped.tsv
brh.npy
brh_features.txt
hayes.npy
hayes.tsv
hayes_features.txt
hayes_remapped.tsv
mielke_spe.tsv


In [7]:
# objects_in_fp = 'data/hayes.tsv'
# objects_np_in_fp = 'data/hayes.npy'
# feature_list_fp = 'data/hayes_features.txt'

objects_in_fp = 'data/bakovic_chart_riggle_hayes.tsv'
objects_np_in_fp = 'data/brh.npy'
feature_list_fp = 'data/brh_features.txt'

In [8]:
objects = prague.load_objects(objects_in_fp)
len(objects)
objects[:3]

symbols = [o['symbol'] for o in objects]
symbols[:10]

94

[OrderedDict([('symbol', 'p'),
              ('syll', '-'),
              ('cons', '+'),
              ('son', '-'),
              ('labial', '+'),
              ('coronal', '-'),
              ('dorsal', '-'),
              ('voice', '-'),
              ('cont', '-'),
              ('del. rel.', '-'),
              ('ant', '0'),
              ('dist', '0'),
              ('strid', '0'),
              ('high', '0'),
              ('low', '0'),
              ('back', '0'),
              ('front', '0'),
              ('approx', '-'),
              ('nas', '-'),
              ('lat', '-'),
              ('s.g.', '-'),
              ('c.g.', '-'),
              ('round', '0'),
              ('ATR', '0')]),
 OrderedDict([('symbol', 'b'),
              ('syll', '-'),
              ('cons', '+'),
              ('son', '-'),
              ('labial', '+'),
              ('coronal', '-'),
              ('dorsal', '-'),
              ('voice', '+'),
              ('cont', '-'),
              ('de

['p', 'b', 'ɸ', 'β', 'f', 'v', 't̪', 'd̪', 'θ', 'ð']

In [9]:
len(objects[0].keys())

24

In [10]:
feature_list = []
with open(feature_list_fp, 'r') as feature_file:
    for feature in feature_file:
        feature_list.append(feature.strip())
len(feature_list)

23

In [11]:
assert len(feature_list) == len(objects[0].keys()) - 1

In [12]:
objects_np = np.load(objects_np_in_fp)
objects_np.shape
objects_np.dtype

(94, 23)

dtype('int8')

In [13]:
assert objects_np.shape[1] == len(feature_list)

In [14]:
assert objects_np.shape[0] == len(objects)

In [15]:
objects[:2]

[OrderedDict([('symbol', 'p'),
              ('syll', '-'),
              ('cons', '+'),
              ('son', '-'),
              ('labial', '+'),
              ('coronal', '-'),
              ('dorsal', '-'),
              ('voice', '-'),
              ('cont', '-'),
              ('del. rel.', '-'),
              ('ant', '0'),
              ('dist', '0'),
              ('strid', '0'),
              ('high', '0'),
              ('low', '0'),
              ('back', '0'),
              ('front', '0'),
              ('approx', '-'),
              ('nas', '-'),
              ('lat', '-'),
              ('s.g.', '-'),
              ('c.g.', '-'),
              ('round', '0'),
              ('ATR', '0')]),
 OrderedDict([('symbol', 'b'),
              ('syll', '-'),
              ('cons', '+'),
              ('son', '-'),
              ('labial', '+'),
              ('coronal', '-'),
              ('dorsal', '-'),
              ('voice', '+'),
              ('cont', '-'),
              ('de

In [16]:
objects_np[:2]

array([[ 0,  0, -1,  0, -1,  1, -1, -1, -1,  0, -1,  0,  0,  1, -1,  0,
        -1,  0, -1, -1,  0, -1, -1],
       [ 0,  0, -1,  0, -1,  1, -1, -1, -1,  0, -1,  0,  0,  1, -1,  0,
        -1,  0, -1, -1,  0, -1,  1]], dtype=int8)

In [17]:
symbol_to_fv = {o['symbol']:objects_np[i]
                for i,o in enumerate(objects)}

In [18]:
objects_np.shape
unique_objects_np = np.unique(objects_np, axis=0)
unique_objects_np.shape

(94, 23)

(91, 23)

In [19]:
fv_to_symbols_map = {prague.HashableArray(fv):{s for s in symbol_to_fv
                                               if np.array_equal(fv, symbol_to_fv[s])}
                     for fv in unique_objects_np}

def fv_to_symbols(fv):
    return fv_to_symbols_map[prague.HashableArray(fv)]

In [20]:
index_to_symbols = [fv_to_symbols(fv) for fv in unique_objects_np]
index_to_symbols

[{'ħ'},
 {'ʕ'},
 {'ə', 'ɜ'},
 {'ɞ'},
 {'ɛ'},
 {'a'},
 {'œ'},
 {'ɶ'},
 {'ɪ'},
 {'ʏ'},
 {'ʌ'},
 {'ɑ'},
 {'ɔ'},
 {'ɒ'},
 {'ʊ'},
 {'ʈ'},
 {'ɖ'},
 {'ɳ'},
 {'ʧ'},
 {'ʤ'},
 {'ʂ'},
 {'ʐ'},
 {'ʃ'},
 {'ʒ'},
 {'ɽ'},
 {'ɭ'},
 {'c'},
 {'ɟ'},
 {'ɲ'},
 {'ç'},
 {'ʝ'},
 {'h'},
 {'ɦ'},
 {'p'},
 {'b'},
 {'m', 'ɱ'},
 {'f'},
 {'v'},
 {'ɸ'},
 {'β'},
 {'ʔ'},
 {'q'},
 {'ɢ'},
 {'k'},
 {'ɡ'},
 {'ɴ'},
 {'ŋ'},
 {'χ'},
 {'ʁ'},
 {'x'},
 {'ɣ'},
 {'j'},
 {'ʎ'},
 {'ʋ'},
 {'ⱱ'},
 {'ʙ'},
 {'w', 'ɥ'},
 {'ʟ'},
 {'ʀ'},
 {'t'},
 {'d'},
 {'t̪'},
 {'d̪'},
 {'n'},
 {'n̪'},
 {'ʦ'},
 {'ʣ'},
 {'s'},
 {'z'},
 {'θ'},
 {'ð'},
 {'ɹ'},
 {'ɻ'},
 {'ɾ'},
 {'l'},
 {'l̪'},
 {'r'},
 {'ɘ'},
 {'ɐ'},
 {'ɵ'},
 {'ɨ'},
 {'ʉ'},
 {'e'},
 {'æ'},
 {'ø'},
 {'i'},
 {'y'},
 {'ɤ'},
 {'o'},
 {'ɯ'},
 {'u'}]

In [21]:
def extension_to_symbols(extension):
    return np.array(index_to_symbols)[extension.nonzero()[0]]

In [22]:
def symbol_to_fd(symbol):
    return [fd for fd in objects if fd['symbol'] == symbol]

symbol_to_fd('i')

[OrderedDict([('symbol', 'i'),
              ('syll', '+'),
              ('cons', '-'),
              ('son', '+'),
              ('labial', '-'),
              ('coronal', '-'),
              ('dorsal', '+'),
              ('voice', '+'),
              ('cont', '+'),
              ('del. rel.', '0'),
              ('ant', '0'),
              ('dist', '0'),
              ('strid', '0'),
              ('high', '+'),
              ('low', '-'),
              ('back', '-'),
              ('front', '+'),
              ('approx', '+'),
              ('nas', '-'),
              ('lat', '-'),
              ('s.g.', '-'),
              ('c.g.', '-'),
              ('round', '-'),
              ('ATR', '+')])]

In [23]:
feature_list

['ATR',
 'ant',
 'approx',
 'back',
 'c.g.',
 'cons',
 'cont',
 'coronal',
 'del. rel.',
 'dist',
 'dorsal',
 'front',
 'high',
 'labial',
 'lat',
 'low',
 'nas',
 'round',
 's.g.',
 'son',
 'strid',
 'syll',
 'voice']

In [24]:
def pfv_to_fd(pfv):
    return prague.feature_vector.to_feature_dict(feature_list, pfv)

random_fd = pfv_to_fd(prague.feature_vector.make_random_pfv(len(feature_list)))
random_fd
random_spe = prague.to_spe(d=random_fd)
random_spe
prague.from_spe(random_spe)
prague.from_spe(random_spe, feature_list)

{'ATR': '+',
 'ant': '0',
 'approx': '+',
 'back': '0',
 'c.g.': '0',
 'cons': '+',
 'cont': '+',
 'coronal': '+',
 'del. rel.': '+',
 'dist': '+',
 'dorsal': '0',
 'front': '0',
 'high': '-',
 'labial': '0',
 'lat': '0',
 'low': '0',
 'nas': '-',
 'round': '+',
 's.g.': '+',
 'son': '-',
 'strid': '+',
 'syll': '+',
 'voice': '0'}

'[+ATR +approx +cons +cont +coronal +del. rel. +dist -high -nas +round +s.g. -son +strid +syll]'

{'ATR': '+',
 'approx': '+',
 'cons': '+',
 'cont': '+',
 'coronal': '+',
 'del. rel.': '+',
 'dist': '+',
 'high': '-',
 'nas': '-',
 'round': '+',
 's.g.': '+',
 'son': '-',
 'strid': '+',
 'syll': '+'}

{'ATR': '+',
 'ant': '0',
 'approx': '+',
 'back': '0',
 'c.g.': '0',
 'cons': '+',
 'cont': '+',
 'coronal': '+',
 'del. rel.': '+',
 'dist': '+',
 'dorsal': '0',
 'front': '0',
 'high': '-',
 'labial': '0',
 'lat': '0',
 'low': '0',
 'nas': '-',
 'round': '+',
 's.g.': '+',
 'son': '-',
 'strid': '+',
 'syll': '+',
 'voice': '0'}

## Combinatoric summary

In [25]:
f"|O| = {len(objects)} distinct object types (including labels)"
f"{unique_objects_np.shape[1]} object features"
"{0:.2E} logically possible object feature vectors".format(2**unique_objects_np.shape[1])
f"|V| = {unique_objects_np.shape[0]} distinct object feature vectors"
"{0:.2E} possible subsets of V".format(2**unique_objects_np.shape[0])
"{0:.2E} possible partial feature vectors".format(3**unique_objects_np.shape[1])

'|O| = 94 distinct object types (including labels)'

'23 object features'

'8.39E+06 logically possible object feature vectors'

'|V| = 91 distinct object feature vectors'

'2.48E+27 possible subsets of V'

'9.41E+10 possible partial feature vectors'

# Identifying a possible Boolean concept

Consider the two sets of objects defined below:

In [26]:
from random import choice

In [27]:
num_observations = 4
random_observation = [choice(objects) for each in range(num_observations)]
lmap(lambda o: o['symbol'],
     random_observation)
random_observation

['ɭ', 'ʣ', 'ʐ', 'v']

[OrderedDict([('symbol', 'ɭ'),
              ('syll', '-'),
              ('cons', '+'),
              ('son', '+'),
              ('labial', '-'),
              ('coronal', '+'),
              ('dorsal', '-'),
              ('voice', '+'),
              ('cont', '-'),
              ('del. rel.', '0'),
              ('ant', '-'),
              ('dist', '-'),
              ('strid', '0'),
              ('high', '0'),
              ('low', '0'),
              ('back', '0'),
              ('front', '0'),
              ('approx', '+'),
              ('nas', '-'),
              ('lat', '+'),
              ('s.g.', '-'),
              ('c.g.', '-'),
              ('round', '0'),
              ('ATR', '0')]),
 OrderedDict([('symbol', 'ʣ'),
              ('syll', '-'),
              ('cons', '+'),
              ('son', '-'),
              ('labial', '-'),
              ('coronal', '+'),
              ('dorsal', '-'),
              ('voice', '+'),
              ('cont', '-'),
              ('de

In [28]:
# matching_symbols = {'t','d','p','b','k','g'} #will probably consume ALL available memory a few cells downstream
matching_symbols = {'j','w'}
non_random_observation = lfilter(lambda o: o['symbol'] in matching_symbols,
                                 objects)
len(non_random_observation)
non_random_observation

2

[OrderedDict([('symbol', 'w'),
              ('syll', '-'),
              ('cons', '-'),
              ('son', '+'),
              ('labial', '+'),
              ('coronal', '-'),
              ('dorsal', '+'),
              ('voice', '+'),
              ('cont', '+'),
              ('del. rel.', '0'),
              ('ant', '0'),
              ('dist', '0'),
              ('strid', '0'),
              ('high', '+'),
              ('low', '-'),
              ('back', '+'),
              ('front', '-'),
              ('approx', '+'),
              ('nas', '-'),
              ('lat', '-'),
              ('s.g.', '-'),
              ('c.g.', '-'),
              ('round', '+'),
              ('ATR', '0')]),
 OrderedDict([('symbol', 'j'),
              ('syll', '-'),
              ('cons', '-'),
              ('son', '+'),
              ('labial', '-'),
              ('coronal', '-'),
              ('dorsal', '+'),
              ('voice', '+'),
              ('cont', '+'),
              ('de

Suppose we didn't know how either set of observations were generated, but that we want to know if all of the examples in each set of observations are instances of at least one Boolean concept (i.e. plausibly generated by/instances of the same defining partial feature vector).

`prague`'s main functionality is to facilitate this kind of calculation and analysis via two functions:

In [29]:
print(prague.get_pfvs_whose_extension_contains.__doc__)


    Given
        a set of observed objects (a stack of feature vectors)
    this returns
        the set of partial feature vectors (a stack, one vector per row)
    whose extension must contain the set of observed objects.
    


In [30]:
print(prague.get_pfvs_whose_extension_is_exactly.__doc__)


    Given
        a set of observed objects (a stack of feature vectors)
        a set of potentially observable objects (another stack of vectors)
    this returns
        the set of partial feature vectors (a stack, one vector per row)
    whose extension must be exactly the set of observed objects.
    


To use these functions, we first (for each set of observations) map each object to its associated NumPy vector representation:

In [36]:
random_observation_pfvs = np.array([symbol_to_fv[o['symbol']]
                                   for o in random_observation])
random_observation_pfvs

array([[ 0, -1,  1,  0, -1,  1, -1,  1,  0, -1, -1,  0,  0, -1,  1,  0,
        -1,  0, -1,  1,  0, -1,  1],
       [ 0,  1, -1,  0, -1,  1, -1,  1,  1, -1, -1,  0,  0, -1, -1,  0,
        -1,  0, -1, -1,  1, -1,  1],
       [ 0, -1, -1,  0, -1,  1,  1,  1,  1, -1, -1,  0,  0, -1, -1,  0,
        -1,  0, -1, -1,  1, -1,  1],
       [ 0,  0, -1,  0, -1,  1,  1, -1,  1, -1, -1,  0,  0,  1, -1,  0,
        -1,  0, -1, -1,  1, -1,  1]], dtype=int8)

Since the random observation set is random from run to run of the notebook, your specific results (if you execute this notebook locally) may vary, but in general, there will be thousands of partial feature vectors (= conjunctive normal form formulas = restricted Boolean concepts) that are compatible with (i.e. could have given rise to) this observation: 

In [37]:
possible_explanations_for_random_observation = prague.get_pfvs_whose_extension_contains(random_observation_pfvs)
possible_explanations_for_random_observation.shape
print(possible_explanations_for_random_observation)
for each_v in possible_explanations_for_random_observation:
    print(prague.to_spe(feature_list, each_v))

(256, 23)

[[ 0  0  0 ...  0 -1  1]
 [ 0  0  0 ...  0 -1  1]
 [ 0  0  0 ...  0 -1  1]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]
[-c.g. +cons -dist -dorsal -nas -s.g. -syll +voice]
[+cons -dist -dorsal -nas -s.g. -syll +voice]
[-c.g. -dist -dorsal -nas -s.g. -syll +voice]
[-c.g. +cons -dorsal -nas -s.g. -syll +voice]
[-c.g. +cons -dist -nas -s.g. -syll +voice]
[-c.g. +cons -dist -dorsal -s.g. -syll +voice]
[-c.g. +cons -dist -dorsal -nas -syll +voice]
[-c.g. +cons -dist -dorsal -nas -s.g. +voice]
[-c.g. +cons -dist -dorsal -nas -s.g. -syll]
[-dist -dorsal -nas -s.g. -syll +voice]
[+cons -dorsal -nas -s.g. -syll +voice]
[+cons -dist -nas -s.g. -syll +voice]
[+cons -dist -dorsal -s.g. -syll +voice]
[+cons -dist -dorsal -nas -syll +voice]
[+cons -dist -dorsal -nas -s.g. +voice]
[+cons -dist -dorsal -nas -s.g. -syll]
[-c.g. -dorsal -nas -s.g. -syll +voice]
[-c.g. -dist -nas -s.g. -syll +voice]
[-c.g. -dist -dorsal -s.g. -syll +voice]
[-c.g. -dist -dorsal -nas -sy

For example:

In [38]:
print(f"Recall: the random observation = \n{lmap(lambda o: o['symbol'], random_observation)}")
a_compatible_concept = choice(possible_explanations_for_random_observation)
print("A possible partial feature vector that could have given rise to this observation:\n"
      f"   {a_compatible_concept}\n"
      f" = {prague.to_spe(feature_list, a_compatible_concept)}"
#       "{0}".format({feature_list[i]:{-1:'-',1:'+'}[v] for i,v in enumerate(a_compatible_concept) if v in {1,-1}})
     )
      
compatible_concept_extension_vector = prague.extension(a_compatible_concept, 
                                                       unique_objects_np)
compatible_concept_extension_as_objects = prague.extension_vector_to_objects(compatible_concept_extension_vector, 
                                                                             unique_objects_np)
compatible_concept_extension_as_symbols = reduce(set.union, 
                                                 lmap(fv_to_symbols, 
                                                      compatible_concept_extension_as_objects), 
                                                 set())
      
print("The total set of observations that exhibit this concept:\n",
      f"{compatible_concept_extension_as_symbols}")

Recall: the random observation = 
['ɭ', 'ʣ', 'ʐ', 'v']
A possible partial feature vector that could have given rise to this observation:
   [ 0  0  0  0 -1  0  0  0  0  0 -1  0  0  0  0  0  0  0 -1  0  0  0  0]
 = [-c.g. -dorsal -s.g.]
The total set of observations that exhibit this concept:
 {'r', 'ʈ', 'ɸ', 'd', 'ʒ', 'ɳ', 'z', 'ɹ', 'ʙ', 'n̪', 'ɱ', 's', 'ʧ', 'ʋ', 't̪', 'f', 'ɖ', 't', 'l', 'ʤ', 'd̪', 'ʣ', 'm', 'ɻ', 'ⱱ', 'ð', 'l̪', 'n', 'v', 'ɾ', 'ɭ', 'ʐ', 'ʦ', 'β', 'p', 'ʃ', 'θ', 'ʂ', 'ɽ', 'b'}


In [39]:
precise_explanations_for_random_observation = prague.get_pfvs_whose_extension_is_exactly(random_observation_pfvs,
                                                                                         objects_np)
precise_explanations_for_random_observation.shape
print(f"{precise_explanations_for_random_observation} = ")
for each_v in precise_explanations_for_random_observation:
    print(prague.to_spe(feature_list, each_v))

#This will usually be empty: most subsets of the set of logically possible objects
# can't be described by a formula in conjunctive normal form

(0, 23)

[] = 


In [40]:
non_random_observation_pfvs = np.array([symbol_to_fv[o['symbol']]
                                        for o in non_random_observation])
print(non_random_observation_pfvs)
for each_v in non_random_observation_pfvs:
    print(prague.to_spe(feature_list, each_v))

[[ 0  0  1  1 -1 -1  1 -1  0  0  1 -1  1  1 -1 -1 -1  1 -1  1  0 -1  1]
 [ 0  0  1 -1 -1 -1  1 -1  0  0  1  1  1 -1 -1 -1 -1 -1 -1  1  0 -1  1]]
[+approx +back -c.g. -cons +cont -coronal +dorsal -front +high +labial -lat -low -nas +round -s.g. +son -syll +voice]
[+approx -back -c.g. -cons +cont -coronal +dorsal +front +high -labial -lat -low -nas -round -s.g. +son -syll +voice]


In [41]:
possible_explanations_for_non_random_observation = prague.get_pfvs_whose_extension_contains(non_random_observation_pfvs)
possible_explanations_for_non_random_observation.shape
print(possible_explanations_for_non_random_observation)
for each_v in possible_explanations_for_non_random_observation:
    print(prague.to_spe(feature_list, each_v))

(16384, 23)

[[ 0  0  1 ...  0 -1  1]
 [ 0  0  0 ...  0 -1  1]
 [ 0  0  1 ...  0 -1  1]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  1 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]
[+approx -c.g. -cons +cont -coronal +dorsal +high -lat -low -nas -s.g. +son -syll +voice]
[-c.g. -cons +cont -coronal +dorsal +high -lat -low -nas -s.g. +son -syll +voice]
[+approx -cons +cont -coronal +dorsal +high -lat -low -nas -s.g. +son -syll +voice]
[+approx -c.g. +cont -coronal +dorsal +high -lat -low -nas -s.g. +son -syll +voice]
[+approx -c.g. -cons -coronal +dorsal +high -lat -low -nas -s.g. +son -syll +voice]
[+approx -c.g. -cons +cont +dorsal +high -lat -low -nas -s.g. +son -syll +voice]
[+approx -c.g. -cons +cont -coronal +high -lat -low -nas -s.g. +son -syll +voice]
[+approx -c.g. -cons +cont -coronal +dorsal -lat -low -nas -s.g. +son -syll +voice]
[+approx -c.g. -cons +cont -coronal +dorsal +high -low -nas -s.g. +son -syll +voice]
[+approx -c.g. -cons +cont -coronal +dorsal +high -lat -nas -s.g. +son -syll +voice]
[+a

[-cons +cont -coronal +dorsal -nas -s.g. +son -syll +voice]
[-cons +cont -coronal +dorsal -low -s.g. +son -syll +voice]
[-cons +cont -coronal +dorsal -low -nas +son -syll +voice]
[-cons +cont -coronal +dorsal -low -nas -s.g. -syll +voice]
[-cons +cont -coronal +dorsal -low -nas -s.g. +son +voice]
[-cons +cont -coronal +dorsal -low -nas -s.g. +son -syll]
[-cons +cont -coronal +dorsal -lat -s.g. +son -syll +voice]
[-cons +cont -coronal +dorsal -lat -nas +son -syll +voice]
[-cons +cont -coronal +dorsal -lat -nas -s.g. -syll +voice]
[-cons +cont -coronal +dorsal -lat -nas -s.g. +son +voice]
[-cons +cont -coronal +dorsal -lat -nas -s.g. +son -syll]
[-cons +cont -coronal +dorsal -lat -low +son -syll +voice]
[-cons +cont -coronal +dorsal -lat -low -s.g. -syll +voice]
[-cons +cont -coronal +dorsal -lat -low -s.g. +son +voice]
[-cons +cont -coronal +dorsal -lat -low -s.g. +son -syll]
[-cons +cont -coronal +dorsal -lat -low -nas -syll +voice]
[-cons +cont -coronal +dorsal -lat -low -nas +son +vo

[+approx -c.g. +cont -coronal +dorsal -low -nas -s.g. +son]
[+approx -c.g. +cont -coronal +dorsal -lat +son -syll +voice]
[+approx -c.g. +cont -coronal +dorsal -lat -s.g. -syll +voice]
[+approx -c.g. +cont -coronal +dorsal -lat -s.g. +son +voice]
[+approx -c.g. +cont -coronal +dorsal -lat -s.g. +son -syll]
[+approx -c.g. +cont -coronal +dorsal -lat -nas -syll +voice]
[+approx -c.g. +cont -coronal +dorsal -lat -nas +son +voice]
[+approx -c.g. +cont -coronal +dorsal -lat -nas +son -syll]
[+approx -c.g. +cont -coronal +dorsal -lat -nas -s.g. +voice]
[+approx -c.g. +cont -coronal +dorsal -lat -nas -s.g. -syll]
[+approx -c.g. +cont -coronal +dorsal -lat -nas -s.g. +son]
[+approx -c.g. +cont -coronal +dorsal -lat -low -syll +voice]
[+approx -c.g. +cont -coronal +dorsal -lat -low +son +voice]
[+approx -c.g. +cont -coronal +dorsal -lat -low +son -syll]
[+approx -c.g. +cont -coronal +dorsal -lat -low -s.g. +voice]
[+approx -c.g. +cont -coronal +dorsal -lat -low -s.g. -syll]
[+approx -c.g. +cont

[-c.g. +cont +dorsal -lat -low -nas -s.g. +voice]
[-c.g. +cont +dorsal -lat -low -nas -s.g. -syll]
[-c.g. +cont +dorsal -lat -low -nas -s.g. +son]
[-c.g. +cont +dorsal +high -s.g. +son -syll +voice]
[-c.g. +cont +dorsal +high -nas +son -syll +voice]
[-c.g. +cont +dorsal +high -nas -s.g. -syll +voice]
[-c.g. +cont +dorsal +high -nas -s.g. +son +voice]
[-c.g. +cont +dorsal +high -nas -s.g. +son -syll]
[-c.g. +cont +dorsal +high -low +son -syll +voice]
[-c.g. +cont +dorsal +high -low -s.g. -syll +voice]
[-c.g. +cont +dorsal +high -low -s.g. +son +voice]
[-c.g. +cont +dorsal +high -low -s.g. +son -syll]
[-c.g. +cont +dorsal +high -low -nas -syll +voice]
[-c.g. +cont +dorsal +high -low -nas +son +voice]
[-c.g. +cont +dorsal +high -low -nas +son -syll]
[-c.g. +cont +dorsal +high -low -nas -s.g. +voice]
[-c.g. +cont +dorsal +high -low -nas -s.g. -syll]
[-c.g. +cont +dorsal +high -low -nas -s.g. +son]
[-c.g. +cont +dorsal +high -lat +son -syll +voice]
[-c.g. +cont +dorsal +high -lat -s.g. -syl

...

[+cont -lat -low -s.g. +son -syll]
[+cont -lat -low -nas -syll +voice]
[+cont -lat -low -nas +son +voice]
[+cont -lat -low -nas +son -syll]
[+cont -lat -low -nas -s.g. +voice]
[+cont -lat -low -nas -s.g. -syll]
[+cont -lat -low -nas -s.g. +son]
[+cont +high -s.g. +son -syll +voice]
[+cont +high -nas +son -syll +voice]
[+cont +high -nas -s.g. -syll +voice]
[+cont +high -nas -s.g. +son +voice]
[+cont +high -nas -s.g. +son -syll]
[+cont +high -low +son -syll +voice]
[+cont +high -low -s.g. -syll +voice]
[+cont +high -low -s.g. +son +voice]
[+cont +high -low -s.g. +son -syll]
[+cont +high -low -nas -syll +voice]
[+cont +high -low -nas +son +voice]
[+cont +high -low -nas +son -syll]
[+cont +high -low -nas -s.g. +voice]
[+cont +high -low -nas -s.g. -syll]
[+cont +high -low -nas -s.g. +son]
[+cont +high -lat +son -syll +voice]
[+cont +high -lat -s.g. -syll +voice]
[+cont +high -lat -s.g. +son +voice]
[+cont +high -lat -s.g. +son -syll]
[+cont +high -lat -nas -syll +voice]
[+cont +high -lat -n

[+approx -lat -nas -s.g. -syll +voice]
[+approx -lat -nas -s.g. +son +voice]
[+approx -lat -nas -s.g. +son -syll]
[+approx -lat -low +son -syll +voice]
[+approx -lat -low -s.g. -syll +voice]
[+approx -lat -low -s.g. +son +voice]
[+approx -lat -low -s.g. +son -syll]
[+approx -lat -low -nas -syll +voice]
[+approx -lat -low -nas +son +voice]
[+approx -lat -low -nas +son -syll]
[+approx -lat -low -nas -s.g. +voice]
[+approx -lat -low -nas -s.g. -syll]
[+approx -lat -low -nas -s.g. +son]
[+approx +high -s.g. +son -syll +voice]
[+approx +high -nas +son -syll +voice]
[+approx +high -nas -s.g. -syll +voice]
[+approx +high -nas -s.g. +son +voice]
[+approx +high -nas -s.g. +son -syll]
[+approx +high -low +son -syll +voice]
[+approx +high -low -s.g. -syll +voice]
[+approx +high -low -s.g. +son +voice]
[+approx +high -low -s.g. +son -syll]
[+approx +high -low -nas -syll +voice]
[+approx +high -low -nas +son +voice]
[+approx +high -low -nas +son -syll]
[+approx +high -low -nas -s.g. +voice]
[+appro

[+dorsal +high -lat -low -syll]
[+dorsal +high -lat -low +son]
[+dorsal +high -lat -low -s.g.]
[+dorsal +high -lat -low -nas]
[-coronal -s.g. +son -syll +voice]
[-coronal -nas +son -syll +voice]
[-coronal -nas -s.g. -syll +voice]
[-coronal -nas -s.g. +son +voice]
[-coronal -nas -s.g. +son -syll]
[-coronal -low +son -syll +voice]
[-coronal -low -s.g. -syll +voice]
[-coronal -low -s.g. +son +voice]
[-coronal -low -s.g. +son -syll]
[-coronal -low -nas -syll +voice]
[-coronal -low -nas +son +voice]
[-coronal -low -nas +son -syll]
[-coronal -low -nas -s.g. +voice]
[-coronal -low -nas -s.g. -syll]
[-coronal -low -nas -s.g. +son]
[-coronal -lat +son -syll +voice]
[-coronal -lat -s.g. -syll +voice]
[-coronal -lat -s.g. +son +voice]
[-coronal -lat -s.g. +son -syll]
[-coronal -lat -nas -syll +voice]
[-coronal -lat -nas +son +voice]
[-coronal -lat -nas +son -syll]
[-coronal -lat -nas -s.g. +voice]
[-coronal -lat -nas -s.g. -syll]
[-coronal -lat -nas -s.g. +son]
[-coronal -lat -low -syll +voice]
[

[-c.g. -cons -coronal -low -nas]
[-c.g. -cons -coronal -lat +voice]
[-c.g. -cons -coronal -lat -syll]
[-c.g. -cons -coronal -lat +son]
[-c.g. -cons -coronal -lat -s.g.]
[-c.g. -cons -coronal -lat -nas]
[-c.g. -cons -coronal -lat -low]
[-c.g. -cons -coronal +high +voice]
[-c.g. -cons -coronal +high -syll]
[-c.g. -cons -coronal +high +son]
[-c.g. -cons -coronal +high -s.g.]
[-c.g. -cons -coronal +high -nas]
[-c.g. -cons -coronal +high -low]
[-c.g. -cons -coronal +high -lat]
[-c.g. -cons -coronal +dorsal +voice]
[-c.g. -cons -coronal +dorsal -syll]
[-c.g. -cons -coronal +dorsal +son]
[-c.g. -cons -coronal +dorsal -s.g.]
[-c.g. -cons -coronal +dorsal -nas]
[-c.g. -cons -coronal +dorsal -low]
[-c.g. -cons -coronal +dorsal -lat]
[-c.g. -cons -coronal +dorsal +high]
[-c.g. -cons +cont -syll +voice]
[-c.g. -cons +cont +son +voice]
[-c.g. -cons +cont +son -syll]
[-c.g. -cons +cont -s.g. +voice]
[-c.g. -cons +cont -s.g. -syll]
[-c.g. -cons +cont -s.g. +son]
[-c.g. -cons +cont -nas +voice]
[-c.g.

[+approx -s.g. +son +voice]
[+approx -s.g. +son -syll]
[+approx -nas -syll +voice]
[+approx -nas +son +voice]
[+approx -nas +son -syll]
[+approx -nas -s.g. +voice]
[+approx -nas -s.g. -syll]
[+approx -nas -s.g. +son]
[+approx -low -syll +voice]
[+approx -low +son +voice]
[+approx -low +son -syll]
[+approx -low -s.g. +voice]
[+approx -low -s.g. -syll]
[+approx -low -s.g. +son]
[+approx -low -nas +voice]
[+approx -low -nas -syll]
[+approx -low -nas +son]
[+approx -low -nas -s.g.]
[+approx -lat -syll +voice]
[+approx -lat +son +voice]
[+approx -lat +son -syll]
[+approx -lat -s.g. +voice]
[+approx -lat -s.g. -syll]
[+approx -lat -s.g. +son]
[+approx -lat -nas +voice]
[+approx -lat -nas -syll]
[+approx -lat -nas +son]
[+approx -lat -nas -s.g.]
[+approx -lat -low +voice]
[+approx -lat -low -syll]
[+approx -lat -low +son]
[+approx -lat -low -s.g.]
[+approx -lat -low -nas]
[+approx +high -syll +voice]
[+approx +high +son +voice]
[+approx +high +son -syll]
[+approx +high -s.g. +voice]
[+approx 

In [42]:
precise_explanations_for_non_random_observation = prague.get_pfvs_whose_extension_is_exactly(non_random_observation_pfvs,
                                                                                             objects_np)
precise_explanations_for_non_random_observation.shape
print(precise_explanations_for_non_random_observation)
for each_v in precise_explanations_for_non_random_observation:
    print(prague.to_spe(feature_list, each_v))

(4672, 23)

[[ 0  0  1 ...  0 -1  1]
 [ 0  0  0 ...  0 -1  1]
 [ 0  0  1 ...  0 -1  1]
 ...
 [ 0  0  0 ...  0 -1  0]
 [ 0  0  0 ...  0 -1  0]
 [ 0  0  0 ...  0 -1  0]]
[+approx -c.g. -cons +cont -coronal +dorsal +high -lat -low -nas -s.g. +son -syll +voice]
[-c.g. -cons +cont -coronal +dorsal +high -lat -low -nas -s.g. +son -syll +voice]
[+approx -cons +cont -coronal +dorsal +high -lat -low -nas -s.g. +son -syll +voice]
[+approx -c.g. +cont -coronal +dorsal +high -lat -low -nas -s.g. +son -syll +voice]
[+approx -c.g. -cons -coronal +dorsal +high -lat -low -nas -s.g. +son -syll +voice]
[+approx -c.g. -cons +cont +dorsal +high -lat -low -nas -s.g. +son -syll +voice]
[+approx -c.g. -cons +cont -coronal +high -lat -low -nas -s.g. +son -syll +voice]
[+approx -c.g. -cons +cont -coronal +dorsal -lat -low -nas -s.g. +son -syll +voice]
[+approx -c.g. -cons +cont -coronal +dorsal +high -low -nas -s.g. +son -syll +voice]
[+approx -c.g. -cons +cont -coronal +dorsal +high -lat -nas -s.g. +son -syll +voice]
[+a

[-c.g. -cons +cont +dorsal +high -lat -low +son -syll]
[-c.g. -cons +cont +dorsal +high -lat -low -s.g. -syll]
[-c.g. -cons +cont +dorsal +high -lat -low -nas -syll]
[-c.g. -cons +cont -coronal -low -s.g. +son -syll +voice]
[-c.g. -cons +cont -coronal -low -nas +son -syll +voice]
[-c.g. -cons +cont -coronal -low -nas -s.g. -syll +voice]
[-c.g. -cons +cont -coronal -low -nas -s.g. +son -syll]
[-c.g. -cons +cont -coronal -lat -low +son -syll +voice]
[-c.g. -cons +cont -coronal -lat -low -s.g. -syll +voice]
[-c.g. -cons +cont -coronal -lat -low -s.g. +son -syll]
[-c.g. -cons +cont -coronal -lat -low -nas -syll +voice]
[-c.g. -cons +cont -coronal -lat -low -nas +son -syll]
[-c.g. -cons +cont -coronal -lat -low -nas -s.g. -syll]
[-c.g. -cons +cont -coronal +high -s.g. +son -syll +voice]
[-c.g. -cons +cont -coronal +high -nas +son -syll +voice]
[-c.g. -cons +cont -coronal +high -nas -s.g. -syll +voice]
[-c.g. -cons +cont -coronal +high -nas -s.g. +son -syll]
[-c.g. -cons +cont -coronal +high

[+approx -cons -coronal +dorsal -lat -low -s.g. -syll]
[+approx -cons -coronal +dorsal -lat -low -nas -syll]
[+approx -cons -coronal +dorsal +high +son -syll +voice]
[+approx -cons -coronal +dorsal +high -s.g. -syll +voice]
[+approx -cons -coronal +dorsal +high -s.g. +son -syll]
[+approx -cons -coronal +dorsal +high -nas -syll +voice]
[+approx -cons -coronal +dorsal +high -nas +son -syll]
[+approx -cons -coronal +dorsal +high -nas -s.g. -syll]
[+approx -cons -coronal +dorsal +high -low -syll +voice]
[+approx -cons -coronal +dorsal +high -low +son -syll]
[+approx -cons -coronal +dorsal +high -low -s.g. -syll]
[+approx -cons -coronal +dorsal +high -low -nas -syll]
[+approx -cons -coronal +dorsal +high -lat -syll +voice]
[+approx -cons -coronal +dorsal +high -lat +son -syll]
[+approx -cons -coronal +dorsal +high -lat -s.g. -syll]
[+approx -cons -coronal +dorsal +high -lat -nas -syll]
[+approx -cons -coronal +dorsal +high -lat -low -syll]
[+approx -cons +cont -low -s.g. +son -syll +voice]


[-c.g. -cons +high -low -nas -syll]
[-c.g. -cons +high -lat -syll +voice]
[-c.g. -cons +high -lat +son -syll]
[-c.g. -cons +high -lat -s.g. -syll]
[-c.g. -cons +high -lat -nas -syll]
[-c.g. -cons +high -lat -low -syll]
[-c.g. -cons +dorsal +son -syll +voice]
[-c.g. -cons +dorsal -s.g. -syll +voice]
[-c.g. -cons +dorsal -s.g. +son -syll]
[-c.g. -cons +dorsal -nas -syll +voice]
[-c.g. -cons +dorsal -nas +son -syll]
[-c.g. -cons +dorsal -nas -s.g. -syll]
[-c.g. -cons +dorsal -low -syll +voice]
[-c.g. -cons +dorsal -low +son -syll]
[-c.g. -cons +dorsal -low -s.g. -syll]
[-c.g. -cons +dorsal -low -nas -syll]
[-c.g. -cons +dorsal -lat -syll +voice]
[-c.g. -cons +dorsal -lat +son -syll]
[-c.g. -cons +dorsal -lat -s.g. -syll]
[-c.g. -cons +dorsal -lat -nas -syll]
[-c.g. -cons +dorsal -lat -low -syll]
[-c.g. -cons +dorsal +high -syll +voice]
[-c.g. -cons +dorsal +high +son -syll]
[-c.g. -cons +dorsal +high -s.g. -syll]
[-c.g. -cons +dorsal +high -nas -syll]
[-c.g. -cons +dorsal +high -low -syll

In [43]:
degree_of_specification_of_exact_matches = np.abs(precise_explanations_for_non_random_observation).sum(axis=1)
minimal_specification_of_exact_matches = np.min(degree_of_specification_of_exact_matches)
print(f"# features specified in exact matches = {minimal_specification_of_exact_matches}")
minimal_pfvs_that_are_exact_matches = precise_explanations_for_non_random_observation[degree_of_specification_of_exact_matches == minimal_specification_of_exact_matches]
print(f"# minimal pfvs that are exact matches = {minimal_pfvs_that_are_exact_matches.shape[0]}")
print(f"Minimal pfvs that are exact matches =\n"
      f"{minimal_pfvs_that_are_exact_matches}")
for each_v in minimal_pfvs_that_are_exact_matches:
    print(prague.to_spe(feature_list, each_v))

# for each_pfv in minimal_pfvs_that_are_exact_matches:
#     print(pfv_to_fd(each_pfv))

print("Sanity check: extensions of the three pfvs = ")
for each_pfv in minimal_pfvs_that_are_exact_matches:
    each_ext = prague.extension(each_pfv, unique_objects_np)
    reduce(set.union, extension_to_symbols(each_ext), set())

# features specified in exact matches = 3
# minimal pfvs that are exact matches = 3
Minimal pfvs that are exact matches =
[[ 0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0 -1  0]
 [ 0  0  0  0  0 -1  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0 -1  0]
 [ 0  0  0  0  0 -1  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0 -1  0]]
[-cons -low -syll]
[-cons +high -syll]
[-cons +dorsal -syll]
Sanity check: extensions of the three pfvs = 


{'j', 'w', 'ɥ'}

{'j', 'w', 'ɥ'}

{'j', 'w', 'ɥ'}

# General case

In [44]:
from tqdm import tqdm

In [45]:
!free -h

zsh:1: command not found: free


In [46]:
#7s on wittgenstein
lower_closures = [prague.lower_closure(o, strict=False) 
                  for o in tqdm(unique_objects_np)]

100%|██████████| 91/91 [00:06<00:00, 14.27it/s]


In [49]:
lower_closures_as_matrix = np.concatenate(lower_closures)
lower_closures_as_matrix.shape
'{:,}'.format(lower_closures_as_matrix.shape[0])

(22765568, 23)

'22,765,568'

In [55]:
#2m on wittgenstein
# all_pfvs_with_nonempty_extension = np.unique(lower_closures_as_matrix, 
#                                              return_index=False,
#                                              axis=0)
# all_pfvs_with_nonempty_extension.shape

# 3s
all_pfvs_with_nonempty_extension_hashed = prague.hash_ternary_pfv(lower_closures_as_matrix)
all_pfvs_with_nonempty_extension_hashed.shape

#4 s
all_pfvs_with_nonempty_extension_set = set(all_pfvs_with_nonempty_extension_hashed)
len(all_pfvs_with_nonempty_extension_set)

(22765568,)

9115112

In [59]:
all_pfvs_with_nonempty_extension = prague.decode_hash(np.array(tuple(all_pfvs_with_nonempty_extension_set), 
                                                               dtype=np.int8), 
                                                      m=len(feature_list))
all_pfvs_with_nonempty_extension.shape

(9115112, 23)

In [60]:
print("# partial feature vectors that pick out a non-empty subset of O: "
      "{0:.2E}\n".format(all_pfvs_with_nonempty_extension.shape[0]),
      "# logically possible partial feature vectors: "
      "{0:.2E}".format(3**all_pfvs_with_nonempty_extension.shape[1]))

# partial feature vectors that pick out a non-empty subset of O: 9.12E+06
 # logically possible partial feature vectors: 9.41E+10


In [61]:
!free -h

zsh:1: command not found: free


In [47]:
# 2m on wittgenstein = no time complexity savings relative to np.unique
# unique_lower_closures = set(lmap(prague.HashableArray,
#                                  list(lower_closures_as_matrix)))

In [48]:
# !free -h

In [49]:
# >>>2m on wittgenstein, uses way too much memory and definitey no savings over
# np.unique
# unique_lower_closures2 = set(lmap(prague.HashableArray,
#                                   cat(map(list, lower_closures_as_matrix))))

In [50]:
# !free -h

In [62]:
#48s on wittgenstein
nonempty_pfv_extensions = prague.extensions(all_pfvs_with_nonempty_extension,
                                            object_inventory=unique_objects_np)

In [63]:
nonempty_pfv_extensions.shape

(9115112, 91)

In [64]:
!free -h

zsh:1: command not found: free


## Demo

In [65]:
a_random_pfv = prague.feature_vector.make_random_pfv(num_features=len(feature_list))
random_extension = prague.extension(a_random_pfv, 
                                    object_inventory=unique_objects_np)
random_objects = prague.extension_vector_to_objects(random_extension, 
                                                    object_inventory=unique_objects_np)
random_symbols = reduce(set.union, extension_to_symbols(random_extension), set())
my_x = random_extension
my_S = random_objects
my_symbs = random_symbols
my_fd = pfv_to_fd(a_random_pfv)
my_symbs_fds = [pfv_to_fd(pfv) for pfv in my_S]


while my_x.sum() == 0:
    a_random_pfv = prague.feature_vector.make_random_pfv(num_features=len(feature_list))
    random_extension = prague.extension(a_random_pfv, 
                                        object_inventory=unique_objects_np)
    random_objects = prague.extension_vector_to_objects(random_extension, 
                                                        object_inventory=unique_objects_np)
    random_symbols = reduce(set.union, extension_to_symbols(random_extension), set())
    my_x = random_extension
    my_S = random_objects
    my_symbs = random_symbols
    my_fd = pfv_to_fd(a_random_pfv)
    my_symbs_fds = [pfv_to_fd(pfv) for pfv in my_S]


print(f"My random PFV:\n{a_random_pfv}\n{prague.to_spe(feature_list, a_random_pfv)}")
# print(f"My random PFV as a feature dict =\n{my_fd}")
print(f"My extension as an 'indicator' vector = \n{my_x}")
print(f"My extension as a stack of {my_S.shape[0]} objects S = \n{my_S}")
print(f"My extension as a set of matching symbols = \n{my_symbs}")
print(f"My extension in SPE-style feature notation:")
for each_v in my_S:
    print(prague.to_spe(feature_list, each_v))
# print(f"My extension as a set of feature dicts = \n",
#       my_symbs_fds)

My random PFV:
[ 0  0  0  0 -1  1  0  1 -1 -1 -1  0  0 -1  0  0 -1  0 -1 -1  0  0 -1]
[-c.g. +cons +coronal -del. rel. -dist -dorsal -labial -nas -s.g. -son -voice]
My extension as an 'indicator' vector = 
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
My extension as a stack of 2 objects S = 
[[ 0 -1 -1  0 -1  1 -1  1 -1 -1 -1  0  0 -1 -1  0 -1  0 -1 -1 -1 -1 -1]
 [ 0  1 -1  0 -1  1 -1  1 -1 -1 -1  0  0 -1 -1  0 -1  0 -1 -1 -1 -1 -1]]
My extension as a set of matching symbols = 
{'t', 'ʈ'}
My extension in SPE-style feature notation:
[-ant -approx -c.g. +cons -cont +coronal -del. rel. -dist -dorsal -labial -lat -nas -s.g. -son -strid -syll -voice]
[+ant -approx -c.g. +cons -cont +coronal -del. rel. -dist -dorsal -labial -lat -nas -s.g. -son -strid -syll -voice]


In [66]:
pfvs_containing_my_S = prague.get_pfvs_whose_extension_contains(my_S)
pfvs_exactly_matching_my_S = prague.get_pfvs_whose_extension_is_exactly(my_S, 
                                                                        object_inventory=unique_objects_np)
specification_of_exact_matches = np.abs(pfvs_exactly_matching_my_S).sum(axis=1)
minimal_specification = np.min(specification_of_exact_matches)
minimal_pfvs_exactly_matching_my_s = pfvs_exactly_matching_my_S[specification_of_exact_matches == minimal_specification]

print("{0:,} PFVs".format(pfvs_containing_my_S.shape[0]),
      f"whose extension contains S = \n{pfvs_containing_my_S}")
print("{0:,} PFVs".format(pfvs_exactly_matching_my_S.shape[0]),
      f"whose extension is exactly S = \n{pfvs_exactly_matching_my_S}")
print("{0:,} PFVs".format(minimal_pfvs_exactly_matching_my_s.shape[0]),
      f"whose extension is exactly S and which are maximally simple"
      f" (i.e. unspecified) = \n{minimal_pfvs_exactly_matching_my_s}\n")
for each_v in minimal_pfvs_exactly_matching_my_s:
    print(prague.to_spe(feature_list, each_v))
print(f"original generating pfv = \n {a_random_pfv}\n "
      f"{prague.to_spe(feature_list, a_random_pfv)}")

65,536 PFVs whose extension contains S = 
[[ 0  0 -1 ... -1 -1 -1]
 [ 0  0  0 ... -1 -1 -1]
 [ 0  0 -1 ... -1 -1 -1]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0 -1 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]
12,288 PFVs whose extension is exactly S = 
[[ 0  0 -1 ... -1 -1 -1]
 [ 0  0  0 ... -1 -1 -1]
 [ 0  0 -1 ... -1 -1 -1]
 ...
 [ 0  0 -1 ...  0  0 -1]
 [ 0  0  0 ... -1  0 -1]
 [ 0  0  0 ...  0  0 -1]]
2 PFVs whose extension is exactly S and which are maximally simple (i.e. unspecified) = 
[[ 0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0  0 -1  0 -1]
 [ 0  0  0  0  0  0  0  0 -1 -1  0  0  0  0  0  0  0  0  0  0  0  0 -1]]

[-dist -strid -voice]
[-del. rel. -dist -voice]
original generating pfv = 
 [ 0  0  0  0 -1  1  0  1 -1 -1 -1  0  0 -1  0  0 -1  0 -1 -1  0  0 -1]
 [-c.g. +cons +coronal -del. rel. -dist -dorsal -labial -nas -s.g. -son -voice]


In [67]:
print(prague.extension(a_random_pfv, unique_objects_np))
for each_exact_match in minimal_pfvs_exactly_matching_my_s:
    print(prague.extension(each_exact_match, unique_objects_np))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [68]:
extension_to_symbols(prague.extension(a_random_pfv, unique_objects_np))
for each_exact_match in minimal_pfvs_exactly_matching_my_s:
    extension_to_symbols(prague.extension(each_exact_match, unique_objects_np))

array([{'ʈ'}, {'t'}], dtype=object)

array([{'ʈ'}, {'t'}], dtype=object)

array([{'ʈ'}, {'t'}], dtype=object)

In [69]:
a_random_pfv
prague.to_spe(feature_list, a_random_pfv)

array([ 0,  0,  0,  0, -1,  1,  0,  1, -1, -1, -1,  0,  0, -1,  0,  0, -1,
        0, -1, -1,  0,  0, -1], dtype=int8)

'[-c.g. +cons +coronal -del. rel. -dist -dorsal -labial -nas -s.g. -son -voice]'

In [70]:
print('Specified indices to modify:')
specified_index_a = choice(list(a_random_pfv.nonzero()[0]))
specified_index_a
specified_index_b = choice(list(a_random_pfv.nonzero()[0]))
while specified_index_b == specified_index_a:
    specified_index_b = choice(list(a_random_pfv.nonzero()[0]))
specified_index_b

print('\nModification vector:')
modification = np.zeros(a_random_pfv.shape, dtype=np.int8)
modification[specified_index_a] = choice([-1, 1])
modification[specified_index_b] = choice([-1, 1])
modification
prague.to_spe(feature_list, modification)

print('\nNaive update:')
updated_naive = prague.feature_vector.spe_update(a_random_pfv, modification)
updated_naive
prague.to_spe(feature_list, updated_naive)

updated = prague.feature_vector.spe_update(a_random_pfv, modification, unique_objects_np)
print('\nMinimally further coerced vectors with non-empty extension:\n'
      f'{updated}')
print('\nCoerced updates:')
if updated.ndim == 1:
    print('No further coercion necessary:')
    print(prague.to_spe(feature_list, updated))
else:
    for each in updated:
        print(prague.to_spe(feature_list, each))

Specified indices to modify:


13

16


Modification vector:


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0], dtype=int8)

'[+labial +nas]'


Naive update:


array([ 0,  0,  0,  0, -1,  1,  0,  1, -1, -1, -1,  0,  0,  1,  0,  0,  1,
        0, -1, -1,  0,  0, -1], dtype=int8)

'[-c.g. +cons +coronal -del. rel. -dist -dorsal +labial +nas -s.g. -son -voice]'


Minimally further coerced vectors with non-empty extension:
[ 0  0  0  0 -1  1  0  1 -1 -1 -1  0  0  1  0  0  1  0 -1 -1  0  0 -1]

Coerced updates:
No further coercion necessary:
[-c.g. +cons +coronal -del. rel. -dist -dorsal +labial +nas -s.g. -son -voice]


## General case redux: BFE comparison

In [71]:
unique_objects_np.shape

random_object = unique_objects_np[choice(np.arange(unique_objects_np.shape[0]))]
random_object

(91, 23)

array([ 1,  0,  1, -1, -1, -1,  1, -1,  0,  0,  1, -1,  1,  1, -1, -1, -1,
        1, -1,  1,  0,  1,  1], dtype=int8)

In [72]:
ro_lc = prague.lower_closure(random_object)

ro_lc.shape

(524288, 23)

In [73]:
ro_lc_bfe = prague.lower_closure_BFE(random_object, None, None)

In [74]:
ro_lc_bfe

{77936108080,
 77936108081,
 46985374957,
 77936108083,
 80806412308,
 80806412309,
 80806412311,
 80806412312,
 46942441960,
 78093746203,
 78093746204,
 80691068957,
 78093746206,
 78093746207,
 80691068960,
 80691068959,
 80691068956,
 81594941476,
 76802949157,
 76802949158,
 81594941477,
 76802949160,
 76802949161,
 81594941480,
 78323384363,
 81594941479,
 78323384365,
 78323384366,
 80806412335,
 80806412336,
 49267343408,
 80806412338,
 80806412339,
 49267343411,
 49267343410,
 49267343407,
 78093746231,
 80691068984,
 78093746233,
 78093746234,
 80691068987,
 80691068986,
 80691068983,
 78093746230,
 81594941503,
 76802949184,
 76802949185,
 81594941504,
 76802949187,
 76802949188,
 81594941507,
 78323384390,
 49381638215,
 78323384392,
 49381638217,
 49381638218,
 49267343435,
 49267343434,
 49267343437,
 49267343438,
 78323384393,
 49310009347,
 78495695986,
 49381638241,
 49381638242,
 78495695987,
 49381638244,
 49381638245,
 80806412390,
 80261152871,
 80806412392,
 80806

In [75]:
ro_lc_hashes = prague.hash_ternary_pfv(ro_lc)
ro_lc_hashes[:10]
ro_lc_hashes_set = set(ro_lc_hashes)
ro_lc_hashes_set == ro_lc_bfe

array([80289732860, 48908673251, 76802948459, 81451994327, 80677153349,
       80418873023, 80246686139, 80304081767, 80289201419, 80289910007],
      dtype=uint64)

True

# Functions

## Motivation

Phonological processes are commonly represented as context-sensitive rewriting rules:

$A \longrightarrow B$ \ $C\_\_D$

...which can be read as "an instance of *A* can rewritten so that it becomes an instance of*B* when the instance occurs between an instance of *C* and an instance of *D*", where all of *A, B, C*, and *D* are either symbols or feature vectors. 

For example, 
[+cons] $\longrightarrow$ [+voice]
is an unconditional rewrite rule that applies to any object (speech sound, symbol) that matches [+cons], and the effect of the rule is to clamp or coerce the `voice` feature to be `+` after application of the rule

For each of the three problems ('Enumeration', 'Compatibility', and 'Exact Match') mentioned in this package's `readme`, there are analogues in the setting of functions. Given a feature system with features *F* (|*F*| = *m*) and a set of possible object types *O* (|*O*| = *n*):

1. *The enumeration problem*: What is the set of functions (defined by two partial feature vectors) such that every function has both a non-empty domain and a non-empty co-domain?
2. *The compatibility problem*: Given a set of (input, output) examples, what is the set of functions whose behavior includes that mapping?
3. *The exact match problem*: Given a set of (input, output) examples, what is the set of functions whose behavior is exactly that mapping?

## Function combinatorics

Suppose one wanted to learn an unconditional rewriting function by observing example mappings. What is the difference in the size of the hypothesis space between assuming a particular feature system (and the assumption that generalizations are always stated or stateable in terms of partial feature vectors)?

In [76]:
my_n = len(objects)
my_n
my_m = len(feature_list)
my_m

94

23

In [77]:
num_total_functions = my_n ** my_n
num_partial_functions = (my_n + 1) ** my_n

print(f'Let 𝚺 denote the alphabet.\n|𝚺| = {my_n}\n')

print("There are {0:.2E} logically possible total functions on 𝚺.".format(num_total_functions))
print("There are {0:.2E} logically possible partial functions on 𝚺.\n".format(num_partial_functions))

num_pfvs = 3 ** my_m
num_pfvs_with_nonempty_ext = all_pfvs_with_nonempty_extension.shape[0]
print(f'The current feature system has {my_m} features.')
print("There are {0:.2E} logically possible partial feature vectors.".format(num_pfvs))
print("There are {0:.2E} partial feature vectors with non-empty extension.\n".format(num_pfvs_with_nonempty_ext))

num_unconditioned_SPE_rewrite_rules = num_pfvs ** 2
num_usrrs_with_nonempty_domain = num_pfvs_with_nonempty_ext * num_pfvs
print("There are {0:.2E} logically possible SPE-style unconditioned rewrite rules.".format(num_unconditioned_SPE_rewrite_rules))
print("There are {0:.2E} such rules with a non-empty domain.".format(num_usrrs_with_nonempty_domain))

Let 𝚺 denote the alphabet.
|𝚺| = 94

There are 2.98E+185 logically possible total functions on 𝚺.
There are 8.05E+185 logically possible partial functions on 𝚺.

The current feature system has 23 features.
There are 9.41E+10 logically possible partial feature vectors.
There are 9.12E+06 partial feature vectors with non-empty extension.

There are 8.86E+21 logically possible SPE-style unconditioned rewrite rules.
There are 8.58E+17 such rules with a non-empty domain.


(Counting how many of the last class of rules have a codomain whose extension is non-empty is not trivial.)